In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'python', 'calc_KVA.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'python', 'calc_KVA.ipynb'], returncode=0)

In [8]:
df_ead_profile = pd.read_csv('SACCR_EAD_profile.csv')
df_ead_profile

,Unnamed: 0,2017/3/31,2017/4/1,2017/4/2,2017/4/3,2017/4/4,2017/4/5,2017/4/6,2017/4/7,2017/4/8,...,2030/3/31,2031/3/31,2032/3/31,2033/3/31,2034/3/31,2035/3/31,2036/3/31,2037/3/31,2042/3/31,2047/3/31
0,EAD_profile,15711322,15711322,22217136,27207840,31414040,35118757,38467159,41545486,44409937,...,613532705,599251055,581478125,560520628,536515463,509620658,479890630,447611857,250219210,0


In [9]:
array_data_grid = df_ead_profile.columns.values.tolist()
del array_data_grid[0]
array_data_grid

['2017/3/31',
 '2017/4/1',
 '2017/4/2',
 '2017/4/3',
 '2017/4/4',
 '2017/4/5',
 '2017/4/6',
 '2017/4/7',
 '2017/4/8',
 '2017/4/9',
 '2017/4/10',
 '2017/4/11',
 '2017/4/12',
 '2017/4/13',
 '2017/4/14',
 '2017/4/15',
 '2017/4/16',
 '2017/4/17',
 '2017/4/18',
 '2017/4/19',
 '2017/4/20',
 '2017/4/21',
 '2017/4/22',
 '2017/4/23',
 '2017/4/24',
 '2017/4/25',
 '2017/4/26',
 '2017/4/27',
 '2017/4/28',
 '2017/4/29',
 '2017/4/30',
 '2017/5/31',
 '2017/6/30',
 '2017/7/31',
 '2017/8/31',
 '2017/9/30',
 '2017/10/31',
 '2017/11/30',
 '2017/12/31',
 '2018/1/31',
 '2018/2/28',
 '2018/3/31',
 '2019/3/31',
 '2020/3/31',
 '2021/3/31',
 '2022/3/31',
 '2023/3/31',
 '2024/3/31',
 '2025/3/31',
 '2026/3/31',
 '2027/3/31',
 '2028/3/31',
 '2029/3/31',
 '2030/3/31',
 '2031/3/31',
 '2032/3/31',
 '2033/3/31',
 '2034/3/31',
 '2035/3/31',
 '2036/3/31',
 '2037/3/31',
 '2042/3/31',
 '2047/3/31']

In [10]:
array_ead = df_ead_profile.iloc[0].values.tolist()
del array_ead[0]
array_ead

[15711322,
 15711322,
 22217136,
 27207840,
 31414040,
 35118757,
 38467159,
 41545486,
 44409937,
 47099549,
 49642749,
 52060998,
 54370971,
 56585931,
 58716630,
 60771932,
 62759245,
 64684837,
 66554064,
 68371550,
 70141317,
 71866890,
 73551381,
 75197550,
 76807861,
 78384525,
 79929532,
 81444685,
 82931619,
 84391827,
 85826672,
 122037335,
 148645272,
 171620647,
 191642132,
 209008149,
 225368475,
 239975398,
 254026851,
 267184833,
 278407620,
 290191781,
 396251017,
 468152060,
 520506931,
 559537802,
 588414489,
 609123498,
 622854620,
 630602836,
 633062091,
 630757694,
 624131552,
 613532705,
 599251055,
 581478125,
 560520628,
 536515463,
 509620658,
 479890630,
 447611857,
 250219210,
 0]

In [22]:
def trapezoidal_rule(array_func_i, array_func_ip1):
    function = (array_func_i + array_func_ip1)/2
    return function

def calc_KVA(capital_cost, risk_weight, alpha, array_EAD_profile, array_grid, netting_effect):
    int_num = len(array_grid)
    maturity_date = array_grid[int_num-1]
    KVA = 0.0
    datetime_obj_maturity = datetime.datetime.strptime(maturity_date, '%Y/%m/%d')
    for i in range(0, int_num -1):
        datetime_obj_date_i = datetime.datetime.strptime(array_grid[i], '%Y/%m/%d')
        datetime_obj_date_i_p1 = datetime.datetime.strptime(array_grid[i+1], '%Y/%m/%d')
        remaining_maturity = ((datetime_obj_maturity - datetime_obj_date_i).days) /365
        DF = (1.0 - np.exp(-0.05*remaining_maturity)/(0.05*remaining_maturity))
        delta_t = ((datetime_obj_date_i_p1 - datetime_obj_date_i).days) /365
        KVA = KVA + capital_cost / alpha * risk_weight * DF * delta_t * trapezoidal_rule(array_ead[i], array_ead[i+1])
    if(netting_effect==True):
        KVA = KVA * 0.5
    else:
        KVA = KVA
    return KVA

In [23]:
calc_KVA(0.6, 0.02, 1.4, array_ead, array_data_grid, False)

36960624.65786349